## What we learn:
+ Cách cấu hình và sử dụng lớp ImageDataGenerator để huấn luyện, xác nhận và kiểm tra tập dữ liệu hình ảnh.
+ Cách sử dụng ImageDataGenerator để chuẩn hóa các giá trị pixel cho hình ảnh đoàn tàu và thử nghiệm
tập dữ liệu.
+ Cách sử dụng ImageDataGenerator để căn giữa và chuẩn hóa các giá trị pixel cho tập dữ liệu hình ảnh đào tạo và kiểm tra.

> 1. MNIST Handwritten Image Classification Dataset.
> 2. ImageDataGenerator Class for Pixel Scaling
> 3. How to Normalize Images With ImageDataGenerator
> 4. How to Center Images With ImageDataGenerator
> 5. How to Standardize Images With ImageDataGenerator

In [9]:
from keras.datasets import mnist
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D
from keras.preprocessing.image import ImageDataGenerator
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# summarize dataset shape
print('Train', x_train.shape, y_train.shape)
print('Test', (x_test.shape, y_test.shape))
# summarize pixel values
print('Train', x_train.min(), x_train.max(), x_train.mean(),
x_train.std())
print('Test', x_test.min(), x_test.max(), x_test.mean(), x_test.std())

Train (60000, 28, 28) (60000,)
Test ((10000, 28, 28), (10000,))
Train 0 255 33.318421449829934 78.56748998339798
Test 0 255 33.791224489795916 79.17246322228644


In [10]:
# reshaping the dataset to have a single channel
width, height, channels = x_train.shape[1], x_train.shape[2], 1 # 28x28x1
x_train= x_train.reshape((x_train.shape[0], width, height, channels))
x_test = x_test.reshape((x_test.shape[0], width, height, channels))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

> Lớp ImageDataGenerator trong Keras cung cấp một bộ kỹ thuật để chia tỷ lệ giá trị pixel trong tập dữ liệu hình ảnh của bạn trước khi tạo mô hình

+ The usage of the ImageDataGenerator class is as follows.
1. Load your dataset.
2. Configure the ImageDataGenerator (e.g. construct an instance).
3. Calculate image statistics (e.g. call the fit() function).
4. Use the generator to fit the model (e.g. pass the instance to the fit generator() function).
5. Use the generator to evaluate the model (e.g. pass the instance to he evaluate generator() function).
> 
1. Tải tập dữ liệu của bạn.
2. Định cấu hình ImageDataGenerator (ví dụ: tạo một phiên bản).
3. Tính toán thống kê hình ảnh (ví dụ: gọi hàm fit ()).
4. Sử dụng trình tạo để phù hợp với mô hình (ví dụ: truyền phiên bản cho hàm fit create ()).
5. Sử dụng trình tạo để đánh giá mô hình (ví dụ: chuyển phiên bản cho
  anh ta đánh giá hàm máy phát ()).

+ Pixel Normalization: scale pixel values to the range 0-1.
+ Pixel Centering: scale pixel values to have a zero mean.
+ Pixel Standardization: scale pixel values to have a zero mean and unit variance.

In [11]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rescale=1.0/255.0, # Normalize Images With ImageDataGenerator
        featurewise_center=True, #Center Images With ImageDataGenerator
        featurewise_std_normalization=True, # Standardize Images With ImageDataGenerator

)

datagen.fit(x_train)
# demonstrate effect on a single batch of samples
train_iterator = datagen.flow(x_train,y_train, batch_size=64)
test_iterator = datagen.flow(x_test, y_test, batch_size=64)
# get a batch
batchX, batchy = train_iterator.next()
print(batchX.shape, batchX.mean())

(64, 28, 28, 1) 0.020053953


In [12]:
# defining the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                 input_shape=(width, height, channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
  
# compiling the  model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
  
# fitting the model with the generator
model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator),
                    epochs=10)
  
# evaluating the model
_, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator),
                                  verbose=0)
print('Test Accuracy: %.3f' % (acc * 100))

Epoch 1/10


/tmp/ipykernel_26924/2239483369.py:17: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator),


  7/938 [..............................] - ETA: 20s - loss: 2.1805 - accuracy: 0.2232

2022-08-06 17:08:42.884532: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.
2022-08-06 17:08:42.884575: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.
2022-08-06 17:08:42.907177: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.
2022-08-06 17:08:42.907284: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.
2022-08-06 17:08:42.930359: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.


938/938 [==============================] - 24s 25ms/step - loss: 0.1529 - accuracy: 0.9538
Epoch 2/10
938/938 [==============================] - 23s 25ms/step - loss: 0.0476 - accuracy: 0.9853
Epoch 3/10
938/938 [==============================] - 23s 25ms/step - loss: 0.0326 - accuracy: 0.9898
Epoch 4/10
938/938 [==============================] - 23s 25ms/step - loss: 0.0257 - accuracy: 0.9921
Epoch 5/10
938/938 [==============================] - 23s 24ms/step - loss: 0.0187 - accuracy: 0.9943
Epoch 6/10
938/938 [==============================] - 23s 25ms/step - loss: 0.0163 - accuracy: 0.9948
Epoch 7/10
938/938 [==============================] - 24s 25ms/step - loss: 0.0124 - accuracy: 0.9959
Epoch 8/10
938/938 [==============================] - 24s 25ms/step - loss: 0.0113 - accuracy: 0.9965
Epoch 9/10
938/938 [==============================] - 23s 25ms/step - loss: 0.0089 - accuracy: 0.9970
Epoch 10/10
938/938 [==============================] - 23s 25ms/step - loss: 0.0090 - accurac

/tmp/ipykernel_26924/2239483369.py:21: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator),


Test Accuracy: 99.120
